In [15]:
# import pandas as pd
import math
# import matplotlib.pyplot as plt
import numpy as np

import gurobipy as gp
from gurobipy import GRB, tuplelist

# import pandapower.networks as pn
# import scipy.io

In [16]:
# m = gp.Model('LinMcCormick')
idx_x = np.arange(1,3,1)
set_x = tuplelist([i for i in idx_x])

In [17]:
def ayy(x):
    a = 2*x
    b = 3*x
    return a,b

a,b = ayy(1)
print(a,b)

2 3


In [18]:
def define_variables(m):
    # X
    x = m.addVars(set_x, name='x')

    for i in [1,2]:
        x[i].LB, x[i].UB = 0, 1.5
        # x[i].Start = 0

    return x

In [19]:
def define_objective(m,x):
    sum_x = -x[1] + x[1]*x[2] - x[2]

    m.setObjective(sum_x, GRB.MINIMIZE)

In [20]:
def define_constraints(m,x):
    m.addConstr(-6*x[1] + 8*x[2] <= 3)
    m.addConstr(3*x[1] - x[2] <= 3)

In [21]:
def mdt(m,x1,x2,p=0,iter=1):
    idx_k = np.arange(0,10,1)
    idx_l = np.arange(p,1,1)

    set_k = tuplelist(idx_k)
    set_l = tuplelist(idx_l)
    set_kl = tuplelist([(i,j) for i in set_k for j in idx_l])

    w12 = m.addVar(lb=-GRB.INFINITY, ub=GRB.INFINITY, name='aux')
    delta_w12 = m.addVar(lb=0, ub=GRB.INFINITY, name='aux_del')
    delta_x1 = m.addVar(lb=0, ub=GRB.INFINITY, name='x_del')
    hat_x1 = m.addVars(set_kl, lb=0, ub=1.5, name='x_hat')
    z = m.addVars(set_kl, vtype=GRB.BINARY, name='z')

    m.addConstr(w12 == gp.quicksum(gp.quicksum(math.pow(10,l)*k*hat_x1[k,l] for k in set_k) for l in set_l) + delta_w12)
    m.addConstr(x1 == gp.quicksum(gp.quicksum(math.pow(10,l)*k*z[k,l] for k in set_k) for l in set_l) + delta_x1)
    m.addConstrs(x2 == gp.quicksum(math.pow(10,l)*k*hat_x1[k,l] for k in set_k) for l in set_l)
    m.addConstrs(hat_x1[k,l] <= 1.5 * z[k,l] for k,l in set_kl)
    m.addConstrs(gp.quicksum(z[k,l] for k in set_k) == 1 for l in set_l)
    # m.addConstr(delta_w12 >= 0)
    m.addConstr(delta_w12 <= 1.5*delta_x1)
    m.addConstr(delta_w12 >= (x2 - 1.5)*math.pow(10,p) + 1.5*delta_x1)
    m.addConstr(delta_w12 <= math.pow(10,p)*x2)
    m.addConstr(delta_x1 <= 1.5*math.pow(10,p))


    return w12,delta_w12,delta_x1,hat_x1,z

def define_objective_mdt(m,x,w):
    sum_x = -x[1] + w - x[2]

    m.setObjective(sum_x, GRB.MINIMIZE)

o = gp.Model('a')
x = define_variables(o)
o.update()

w,del_w,del_x,h_x,zz = [],[],[],[],[]
w12,delta_w12,delta_x1,hat_x1,z = mdt(o,x[1],x[2],p=-1)
define_constraints(o,x)

define_objective(o,x,w12)

o.write('MDT.lp')

o.optimize()
o.NumConstrs

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (26120.2))

CPU model: AMD Ryzen 7 2700X Eight-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 32 rows, 45 columns and 132 nonzeros
Model fingerprint: 0xe9920a26
Variable types: 25 continuous, 20 integer (20 binary)
Coefficient statistics:
  Matrix range     [1e-01, 9e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 2e+00]
  RHS range        [2e-01, 3e+00]
Found heuristic solution: objective 0.0000000
Presolve removed 30 rows and 33 columns
Presolve time: 0.00s
Presolved: 2 rows, 12 columns, 21 nonzeros
Variable types: 2 continuous, 10 integer (10 binary)

Root relaxation: objective -1.000000e+00, 1 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0  

32

In [22]:
-x[1].X + x[1].X * x[2].X - x[2].X

-1.0

In [23]:
m = gp.Model('P1')
x = define_variables(m)
m.update()
define_objective(m,x)
define_constraints(m,x)

# m.Params.NonConvex = 2
# m.Params.PStart = [0,0]
# m.Params.SolutionLimit = 1
m.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (26120.2))

CPU model: AMD Ryzen 7 2700X Eight-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 2 rows, 2 columns and 4 nonzeros
Model fingerprint: 0x5c271958
Model has 1 quadratic objective term
Coefficient statistics:
  Matrix range     [1e+00, 8e+00]
  Objective range  [1e+00, 1e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [2e+00, 2e+00]
  RHS range        [3e+00, 3e+00]

Continuous model is non-convex -- solving as a MIP

Found heuristic solution: objective 0.0000000
Found heuristic solution: objective -0.7500000
Presolve time: 0.00s
Presolved: 4 rows, 3 columns, 8 nonzeros
Presolved model has 1 bilinear constraint(s)
Variable types: 3 continuous, 0 integer (0 binary)
Found heuristic solution: objective -1.0833333

Root relaxation: objective -1.500000e+00, 1 iterations, 0.00 seconds (0.00 work units)

In [27]:
x
m.Runtime

0.0710000991821289